# EV Parameter Sweep Analysis

This notebook visualises the aggregated output produced by `scripts/generate_ev_case_study.py`.

## Usage
1. Run the sweep script to produce `analysis/ev_param_sweep.json` and `analysis/ev_param_sweep.csv`.
2. Update `JSON_PATH` / `CSV_PATH` below if you stored the results elsewhere.
3. Execute the cells to inspect performance heatmaps and recommended parameter ranges.


In [ ]:
from pathlib import Path
import json

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

JSON_PATH = Path('analysis/ev_param_sweep.json')
CSV_PATH = Path('analysis/ev_param_sweep.csv')

results = json.loads(JSON_PATH.read_text())
df = pd.read_csv(CSV_PATH)
display(df.head())


In [ ]:
# Example heatmap: win rate across threshold & decay
if {'param.threshold_lcb', 'param.decay'}.issubset(df.columns):
    pivot = df.pivot_table(index='param.threshold_lcb', columns='param.decay', values='derived.win_rate')
    plt.figure(figsize=(8, 6))
    sns.heatmap(pivot, annot=True, fmt='.2f', cmap='viridis')
    plt.title('Win Rate heatmap')
    plt.ylabel('threshold_lcb')
    plt.xlabel('decay')
else:
    print('Add --threshold and --decay sweeps to enable this view.')


In [ ]:
# Recommend parameter windows by filtering on win rate / pips per trade thresholds
WIN_RATE_MIN = 0.52
PIPS_PER_TRADE_MIN = 0.0

filtered = df[(df['derived.win_rate'] >= WIN_RATE_MIN) & (df['derived.pips_per_trade'] >= PIPS_PER_TRADE_MIN)]
if not filtered.empty:
    display(filtered.sort_values('derived.win_rate', ascending=False))
else:
    print('No parameter combination passed the thresholds; adjust WIN_RATE_MIN / PIPS_PER_TRADE_MIN.')
